# File and Directory Operations

**Duration:** 25 minutes  
**Level:** Beginner

Master all file and directory operations in genro-storage.

## What You'll Learn

- Reading and writing files (text and binary)
- File properties and metadata
- Directory creation and navigation
- Copying and moving files
- Deleting files and directories
- Working with paths
- Context managers and streaming

Let's dive in! 📁

In [ ]:
from genro_storage import StorageManager
import tempfile
import os

# Setup
storage = StorageManager()
temp_dir = tempfile.mkdtemp()

storage.configure([
    {'name': 'mem', 'type': 'memory'},
    {'name': 'local', 'type': 'local', 'path': temp_dir}
])

print("✓ Storage configured")

## 1. Reading and Writing Text

The simplest operations:

In [ ]:
# Write text
note = storage.node('mem:note.txt')
note.write('Hello, World!\nThis is a test.')

# Read text
content = note.read()
print("Content:")
print(content)

# Write with different encoding
italian = storage.node('mem:italian.txt')
italian.write('Ciao! Come stai? È bello!', encoding='utf-8')

print(f"\n✓ Text operations complete")
print(f"Note size: {note.size} bytes")
print(f"Italian size: {italian.size} bytes")

## 2. Reading and Writing Binary Data

For non-text files (images, videos, etc.):

In [ ]:
# Simulate image data
fake_image = b'\x89PNG\r\n\x1a\n' + b'\x00' * 100

# Write binary
image = storage.node('mem:photo.png')
image.write(fake_image, mode='wb')

# Read binary
data = image.read(mode='rb')

print(f"✓ Binary file written")
print(f"Size: {image.size} bytes")
print(f"First 8 bytes: {data[:8]}")
print(f"Is PNG: {data.startswith(b'\x89PNG')}")

## 3. File Properties

Access metadata about files:

In [ ]:
doc = storage.node('mem:document.pdf')
doc.write('PDF content here...')

print("File Properties:")
print(f"  Full path: {doc.fullpath}")
print(f"  Basename: {doc.basename}")
print(f"  Stem: {doc.stem}")
print(f"  Suffix: {doc.suffix}")
print(f"  Size: {doc.size} bytes")
print(f"  Modified time: {doc.mtime}")
print(f"  Exists: {doc.exists}")
print(f"  Is file: {doc.isfile}")
print(f"  Is directory: {doc.isdir}")
print(f"  MIME type: {doc.mimetype}")
print(f"  MD5 hash: {doc.md5hash}")

## 4. Checking File Existence

Before reading or writing:

In [ ]:
file1 = storage.node('mem:existing.txt')
file1.write('I exist!')

file2 = storage.node('mem:nonexistent.txt')

print(f"file1 exists: {file1.exists}")
print(f"file2 exists: {file2.exists}")

# Safe reading
if file1.exists:
    print(f"\nContent: {file1.read()}")
else:
    print("\nFile does not exist!")

# Try reading non-existent (will raise error)
try:
    file2.read()
except FileNotFoundError as e:
    print(f"\n✓ Error caught: {e}")

## 5. Creating Directories

Organize files in directories:

In [ ]:
# Simple directory
docs = storage.node('mem:documents')
docs.mkdir()

print(f"Directory created: {docs.exists} and {docs.isdir}")

# Nested directories with parents=True
reports = storage.node('mem:company/reports/2024/q4')
reports.mkdir(parents=True)

print(f"Nested directory created: {reports.exists}")

# With exist_ok to avoid errors
docs.mkdir(exist_ok=True)  # Doesn't raise error

print("✓ Directory operations complete")

## 6. Navigating Paths

Move through directory structure:

In [ ]:
# Create a file in a directory
report_file = storage.node('mem:company/reports/2024/q4/sales.txt')
report_file.parent.mkdir(parents=True, exist_ok=True)
report_file.write('Q4 Sales: $1M')

# Navigate using parent
print(f"File: {report_file.fullpath}")
print(f"Parent: {report_file.parent.fullpath}")
print(f"Grandparent: {report_file.parent.parent.fullpath}")

# Navigate using child
company = storage.node('mem:company')
q4_dir = company.child('reports', '2024', 'q4')
print(f"\nNavigated to: {q4_dir.fullpath}")

# Access file through navigation
sales = q4_dir.child('sales.txt')
print(f"Sales content: {sales.read()}")

## 7. Listing Directory Contents

Explore what's in a directory:

In [ ]:
# Create some files
proj_dir = storage.node('mem:project')
proj_dir.mkdir()

proj_dir.child('README.md').write('# Project')
proj_dir.child('main.py').write('print("hello")')
proj_dir.child('config.json').write('{"debug": true}')

# Create subdirectory
src_dir = proj_dir.child('src')
src_dir.mkdir()
src_dir.child('app.py').write('# Main app')

# List all children
print("Project contents:")
for child in proj_dir.children():
    type_str = 'DIR' if child.isdir else 'FILE'
    size_str = f"({child.size} bytes)" if child.isfile else ""
    print(f"  [{type_str}] {child.basename} {size_str}")

## 8. Copying Files

Duplicate files and directories:

In [ ]:
# Simple file copy
original = storage.node('mem:original.txt')
original.write('Original content')

copy = storage.node('mem:copy.txt')
original.copy_to(copy)

print(f"✓ File copied")
print(f"Original: {original.read()}")
print(f"Copy: {copy.read()}")

# Copy to different backend
local_copy = storage.node('local:saved.txt')
original.copy_to(local_copy)

print(f"\n✓ Copied to local filesystem")
print(f"Path: {local_copy.fullpath}")

# Copy directory recursively
backup_dir = storage.node('mem:project_backup')
proj_dir.copy_to(backup_dir)

print(f"\n✓ Directory copied")
print(f"Backup contains {len(list(backup_dir.children()))} items")

## 9. Moving Files

Move (rename) files:

In [ ]:
# Create a file
temp_file = storage.node('mem:temp_name.txt')
temp_file.write('Content to move')

print(f"Before move: {temp_file.fullpath} exists={temp_file.exists}")

# Move to new location
final_file = storage.node('mem:final_name.txt')
temp_file.move(final_file)

print(f"After move: {temp_file.fullpath} exists={temp_file.exists}")
print(f"New location: {final_file.fullpath} exists={final_file.exists}")
print(f"Content preserved: {final_file.read()}")

# Note: temp_file is automatically updated to point to new location
print(f"\ntemp_file now points to: {temp_file.fullpath}")

## 10. Deleting Files and Directories

Remove files when no longer needed:

In [ ]:
# Create files to delete
trash1 = storage.node('mem:delete_me.txt')
trash1.write('Delete this')

print(f"Before delete: {trash1.exists}")

# Delete file
trash1.delete()

print(f"After delete: {trash1.exists}")

# Delete directory (recursive)
trash_dir = storage.node('mem:trash_dir')
trash_dir.mkdir()
trash_dir.child('file1.txt').write('trash')
trash_dir.child('file2.txt').write('trash')

print(f"\nDirectory exists: {trash_dir.exists}")
trash_dir.delete()
print(f"After delete: {trash_dir.exists}")

## 11. Context Managers (with statement)

Use familiar Python file operations:

In [ ]:
log = storage.node('mem:app.log')

# Write mode
with log.open('w') as f:
    f.write('Application started\n')
    f.write('Loading configuration...\n')
    f.write('Ready!\n')

# Read mode
with log.open('r') as f:
    for line in f:
        print(f"  LOG: {line.strip()}")

# Append mode
with log.open('a') as f:
    f.write('New log entry\n')

print(f"\n✓ Log has {log.size} bytes")

## 12. Working with Large Files

Stream data for memory efficiency:

In [ ]:
# Create a "large" file
large = storage.node('mem:large.txt')

# Write in chunks
with large.open('w') as f:
    for i in range(1000):
        f.write(f"Line {i}: This is some data\n")

print(f"Large file created: {large.size} bytes")

# Read in chunks
line_count = 0
with large.open('r') as f:
    for line in f:
        line_count += 1

print(f"Lines in file: {line_count}")

# Read specific number of bytes
with large.open('rb') as f:
    first_100 = f.read(100)
    print(f"\nFirst 100 bytes: {first_100[:50]}...")

## 13. Conditional Writing

Only write if content changed:

In [ ]:
config = storage.node('mem:config.txt')
config.write('version=1.0')

# Try writing same content
changed = config.write('version=1.0', skip_if_unchanged=True)
print(f"Content changed: {changed}")

# Write different content
changed = config.write('version=2.0', skip_if_unchanged=True)
print(f"Content changed: {changed}")

print(f"\nFinal content: {config.read()}")

## 14. File Comparison

Compare files by content:

In [ ]:
file_a = storage.node('mem:a.txt')
file_a.write('Same content')

file_b = storage.node('mem:b.txt')
file_b.write('Same content')

file_c = storage.node('mem:c.txt')
file_c.write('Different content')

# Compare by hash
print(f"A and B same: {file_a.md5hash == file_b.md5hash}")
print(f"A and C same: {file_a.md5hash == file_c.md5hash}")

# Compare by size (faster)
print(f"\nA and B size match: {file_a.size == file_b.size}")
print(f"A and C size match: {file_a.size == file_c.size}")

# Using == operator
print(f"\nA == B: {file_a == file_b}")
print(f"A == C: {file_a == file_c}")

## 15. Try It Yourself! 🎯

**Exercise 1:** Create a directory structure for a blog:
```
blog/
  posts/
    2024/
      01-first-post.md
      02-second-post.md
  assets/
    images/
    css/
```

In [ ]:
# Your code here


**Exercise 2:** Write a function that counts total files and directories:

In [ ]:
def count_items(directory):
    """Count files and directories recursively"""
    # Your code here
    pass

# Test it
# blog = storage.node('mem:blog')
# files, dirs = count_items(blog)
# print(f"Files: {files}, Directories: {dirs}")

**Exercise 3:** Create a backup function that copies with timestamp:

In [ ]:
from datetime import datetime

def backup_with_timestamp(source_node, backup_dir):
    """
    Copy file to backup_dir with timestamp in name.
    Example: file.txt -> file_20240115_143022.txt
    """
    # Your code here
    pass

# Test it
# important = storage.node('mem:important.txt')
# important.write_text('Critical data')
# backup_dir = storage.node('mem:backups')
# backup_dir.mkdir()
# backup_with_timestamp(important, backup_dir)

## 16. Cleanup

In [ ]:
import shutil

if os.path.exists(temp_dir):
    shutil.rmtree(temp_dir)

print("✓ Cleanup complete")

## Summary

You've mastered:

- ✓ Reading and writing text and binary files
- ✓ File properties (size, mtime, type, hash)
- ✓ Creating and navigating directories
- ✓ Listing directory contents
- ✓ Copying, moving, and deleting files
- ✓ Context managers for file operations
- ✓ Streaming large files
- ✓ Conditional writes and file comparison

## Key Methods

**Reading:**
- `read()`, `read(mode='rb')`, `open('r')`

**Writing:**
- `write()`, `write(mode='wb')`, `open('w')`

**Properties:**
- `exists`, `isfile`, `isdir`, `size`, `mtime`, `basename`, `suffix`

**Navigation:**
- `parent`, `child()`, `children()`

**Operations:**
- `mkdir()`, `copy()`, `move()`, `delete()`

## What's Next?

Ready for advanced features? Continue to:

- **[04_virtual_nodes.ipynb](04_virtual_nodes.ipynb)** - Learn about iternode, diffnode, and zip
- **[05_copy_strategies.ipynb](05_copy_strategies.ipynb)** - Smart copying with skip strategies

Happy file handling! 📂